# Load packages

In [3]:
!pip uninstall -y kfp
!pip install kfp

Found existing installation: kfp 1.0.4
Uninstalling kfp-1.0.4:
  Successfully uninstalled kfp-1.0.4
  Using cached kfp-1.4.0-py3-none-any.whl
  Using cached kfp_server_api-1.5.0-py3-none-any.whl
  Using cached kfp_pipeline_spec-0.1.7-py3-none-any.whl (27 kB)
  Using cached docstring_parser-0.7.3-py3-none-any.whl
  Using cached fire-0.4.0-py2.py3-none-any.whl
  Using cached termcolor-1.1.0-py3-none-any.whl
  Attempting uninstall: kfp-server-api
    Found existing installation: kfp-server-api 1.0.4
    Uninstalling kfp-server-api-1.0.4:
      Successfully uninstalled kfp-server-api-1.0.4


In [4]:
import kfp
import json
from kfp.onprem import use_k8s_secret
import os
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler

kfp.__version__

/opt/conda/lib/python3.8/site-packages/kubernetes/utils/create_from_yaml.py:77: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if kind is not "":


'1.4.0'

# Enter your gateway loadbalancer and the token from the cookie
[Use this extension on chrome to get token]( https://chrome.google.com/webstore/detail/editthiscookie/fngmhnnpilhplaeedifhccceomclgfbg?hl=en)

![image.png](./image.png)

In [7]:
#Update values for the load balancer and auth session
HOST='istio-ingressgateway.istio-system.svc.cluster.local'
AUTH="authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh"
namespace="kubeflow-user-example-com"

In [8]:
client = kfp.Client(host=HOST+"/pipeline", cookies=AUTH)
experiments = client.list_experiments(namespace=namespace)
my_experiment = experiments.experiments[0]
my_experiment

{'created_at': datetime.datetime(2021, 4, 22, 8, 44, 39, tzinfo=tzlocal()),
 'description': None,
 'id': 'aac96a63-616e-4d88-9334-6ca8df2bb956',
 'name': 'Default',
 'resource_references': [{'key': {'id': 'kubeflow-user-example-com',
                                  'type': 'NAMESPACE'},
                          'name': None,
                          'relationship': 'OWNER'}],
 'storage_state': 'STORAGESTATE_AVAILABLE'}

### Using dsl for model archiver and torchserve integration
#### Init container is used for mar file generation

In [25]:
DEPLOY="torchserve"
MODEL="cifar10"

In [26]:
prepare_tensorboard_op = load_component_from_file("./examples/cifar10/yaml/tensorboard/component.yaml")
prep_op = components.load_component_from_file(
    "./examples/cifar10/yaml/pre_process/component.yaml"
)
train_op = components.load_component_from_file(
    "./examples/cifar10/yaml/train/component.yaml"
)
deploy_op = load_component_from_file("./examples/cifar10/yaml/deploy/component.yaml")

In [27]:
minio_op = components.load_component_from_file(
    "./examples/cifar10/yaml/minio/component.yaml"
)

In [28]:
@dsl.pipeline(name="Training pipeline", description="Sample training job test")
def pytorch_cifar10(minio_endpoint='minio-service.kubeflow:9000',
    log_bucket='mlpipeline',
    log_dir=f'tensorboard/logs/{dsl.RUN_ID_PLACEHOLDER}',
    mar_path=f'mar/{dsl.RUN_ID_PLACEHOLDER}/model-store',
    config_prop_path=f'mar/{dsl.RUN_ID_PLACEHOLDER}/config',
    model_uri = f's3://mlpipeline/mar/{dsl.RUN_ID_PLACEHOLDER}',
    tf_image='gcr.io/deeplearning-platform-release/tf2-cpu.2-3:latest'):

    @dsl.component
    def ls(input_dir: str):
        return dsl.ContainerOp(
            name='list',
            image='busybox:latest',
            command=["ls", "-R", "%s" % input_dir]
        )
    
    prepare_tb_task = prepare_tensorboard_op(
        log_dir_uri=f's3://{log_bucket}/{log_dir}',
        image=tf_image,
        pod_template_spec=json.dumps({
            'spec': {
                'containers': [{
                    'env': [{
                        'name': 'AWS_ACCESS_KEY_ID',
                        'valueFrom': {
                            'secretKeyRef': {
                                'name': 'mlpipeline-minio-artifact',
                                'key': 'accesskey'
                            }
                        }
                    }, {
                        'name': 'AWS_SECRET_ACCESS_KEY',
                        'valueFrom': {
                            'secretKeyRef': {
                                'name': 'mlpipeline-minio-artifact',
                                'key': 'secretkey'
                            }
                        }
                    }, {
                        'name': 'AWS_REGION',
                        'value': 'minio'
                    }, {
                        'name': 'S3_ENDPOINT',
                        'value': f'{minio_endpoint}',
                    }, {
                        'name': 'S3_USE_HTTPS',
                        'value': '0',
                    }, {
                        'name': 'S3_VERIFY_SSL',
                        'value': '0',
                    }]
                }],
            },
        })
    ).set_display_name("Visualization")
    
    prep_task=prep_op().after(prepare_tb_task).set_display_name("Preprocess & Transform")
    train_task=train_op(input_data=prep_task.outputs['output_data']).after(prep_task).set_display_name("Training")
    
    minio_tb_upload = minio_op( bucket_name="mlpipeline", folder_name=log_dir, input_path=train_task.output         ["tensorboard_root"], filename="",) .apply( use_k8s_secret( secret_name="mlpipeline-minio-artifact", k8s_secret_key_to_env={ "secretkey": "MINIO_SECRET_KEY", "accesskey": "MINIO_ACCESS_KEY", },)) .after(train_task) .set_display_name("Tensorboard Events Pusher")) 
    minio_mar_upload = ( minio_op( bucket_name="mlpipeline", folder_name=mar_path, input_path=train_task.outputs["checkpoint_dir"], filename="cifar10_test.mar",) .apply( use_k8s_secret( secret_name="mlpipeline-minio-artifact", k8s_secret_key_to_env={ "secretkey": "MINIO_SECRET_KEY", "accesskey": "MINIO_ACCESS_KEY", },)) .after(train_task) .set_display_name("Mar Pusher")) 
    minio_config_upload = ( minio_op( bucket_name="mlpipeline", folder_name=config_prop_path, input_path=train_task.outputs["checkpoint_dir"], filename="config.properties",) .apply( use_k8s_secret( secret_name="mlpipeline-minio-artifact", k8s_secret_key_to_env={ "secretkey": "MINIO_SECRET_KEY", "accesskey": "MINIO_ACCESS_KEY", },)) .after(train_task) .set_display_name("Conifg Pusher"))
    
    model_uri= str(model_uri)
    isvc_yaml = '''
    apiVersion: "serving.kubeflow.org/v1beta1"
    kind: "InferenceService"
    metadata:
      name: {}
      namespace: {}
    spec:
      predictor:
        serviceAccountName: sa
        pytorch:
          storageUri: {}
          resources:
            limits:
              memory: 4Gi   
    '''.format(DEPLOY, namespace, model_uri)
    deploy_task = deploy_op(
        action='apply',
        inferenceservice_yaml=isvc_yaml
    ).after(minio_mar_upload).set_display_name("Deployer")
    

In [29]:
# Compile pipeline
compiler.Compiler().compile(pytorch_cifar10, 'pytorch.tar.gz', type_check=True)

In [30]:
# Execute pipeline
run = client.run_pipeline(my_experiment.id, 'pytorch-cifar10', 'pytorch.tar.gz')

### Wait for inference service below to go to `READY True` state.

In [31]:
!kubectl get isvc $DEPLOY

NAME         URL                                                       READY   PREV   LATEST   PREVROLLEDOUTREVISION   LATESTREADYREVISION                  AGE
torchserve   http://torchserve.kubeflow-user-example-com.example.com   True           100                              torchserve-predictor-default-rdjcb   7d


# Inference

In [32]:
!kubectl get isvc $DEPLOY -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3

bertserve.kubeflow-user-example-com.example.com


In [33]:
S_HOSTNAME=!kubectl get isvc $DEPLOY  -o jsonpath='{.items[0].status.url}' | cut -d "/" -f 3
SERVICE_HOSTNAME=S_HOSTNAME[0]
SERVICE_HOSTNAME

'bertserve.kubeflow-user-example-com.example.com'

# Prediction

In [35]:
!curl -v -H "Host: torchserve.kubeflow-user-example-com.example.com" -H "Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh" "http://istio-ingressgateway.istio-system.svc.cluster.local/v1/models/cifar10:predict" -d @./examples/cifar10/input.json

*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/cifar10:predict HTTP/1.1
> Host: torchserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh
> Content-Length: 148025
> Content-Type: application/x-www-form-urlencoded
> Expect: 100-continue
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 151
< content-type: application/json; charset=UTF-8
< date: Thu, 06 May 2021 17:18:53 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 780
< 
* Connection #0 to host istio-ingressgateway.istio-system.svc.cluster.local left intact
{"p

In [36]:
!curl -v -H "Host: torchserve.kubeflow-user-example-com.example.com" -H "Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh" "http://istio-ingressgateway.istio-system.svc.cluster.local/v1/models/cifar10:explain" -d @./examples/cifar10/input.json

*   Trying 10.100.251.14:80...
* TCP_NODELAY set
* Connected to istio-ingressgateway.istio-system.svc.cluster.local (10.100.251.14) port 80 (#0)
> POST /v1/models/cifar10:explain HTTP/1.1
> Host: torchserve.kubeflow-user-example-com.example.com
> User-Agent: curl/7.68.0
> Accept: */*
> Cookie: authservice_session=MTYyMDMxNjgzMHxOd3dBTkZSQldVcEZSVVJUTkRaWlZGWkVVRU5YUjFsTVRGRkZURU5hV1Znek5WUlVWVUUzV0RWRk5WWkdVbFpLTWtwVFJUWmFUMUU9fHBYwVCYxqRB5nYnc6qypZYmZBB63tgI6ul1IJGcuerh
> Content-Length: 148025
> Content-Type: application/x-www-form-urlencoded
> Expect: 100-continue
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 100 Continue
* We are completely uploaded and fine
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< content-length: 3484529
< content-type: application/json; charset=UTF-8
< date: Thu, 06 May 2021 17:19:12 GMT
< server: istio-envoy
< x-envoy-upstream-service-time: 11519
< 
{"explanations": [[[[0.00045112044775389034, 0.004812065252169265, 0.0058242244681941

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

